In [ ]:
docs = [
    "나는 너 만나 사과를 먹으면서 너에게 잘못했던 문제에 대해서 사과를 했어",
    "나는 사과를 먹으면서 너와 즐겁게 대화를 했어",
    "나는 오늘 데이트를 했는데 사과나무를 봤어",
    "어제 바나나를 먹었어",
    "여름에 바나나보트를 탔어"
]

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import time

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')



device: cuda


In [ ]:
def load_model_with_retry(model_name, max_retries=3):
    """재시도 로직이 포함된 모델 로딩"""
    for attempt in range(max_retries):
        try:
            print(f"모델 다운로드 시도 {attempt + 1}/{max_retries}...")
            model = SentenceTransformer(model_name)
            print("✓ 모델 로딩 성공!")
            return model
        except Exception as e:
            print(f"✗ 시도 {attempt + 1} 실패: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)  # 5초 대기 후 재시도
            else:
                raise

In [ ]:
model_candidates = [
    "jhgan/ko-sroberta-multitask",
    "BM-K/KoSimCSE-roberta-multitask",
    "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
]

model = None


# model = SentenceTransformer("jhgan/ko-sroberta-multitask").to(device)
# model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2").to(device)
# model = SentenceTransformer("bert-base-multilingual-cased").to(device)


In [ ]:
for model_name in model_candidates:
    try:
        print(f"\n'{model_name}' 로딩 시도 중...")
        model = load_model_with_retry(model_name)
        print(f"사용 모델: {model_name}")
        break
    except Exception as e:
        print(f"'{model_name}' 로딩 실패, 다음 모델 시도...")
        continue

if model is None:
    raise Exception("모든 모델 로딩 실패")

model = model.to(device)


'jhgan/ko-sroberta-multitask' 로딩 시도 중...
모델 다운로드 시도 1/3...
✓ 모델 로딩 성공!
사용 모델: jhgan/ko-sroberta-multitask


In [ ]:
doc_emb = model.encode(docs,convert_to_tensor=True, normalize_embeddings=True).to(device)

In [ ]:
def search(query:str,top_k:int=2):
  q_emb = model.encode(query,convert_to_tensor=True, normalize_embeddings=True).to(device)
  scores = util.cos_sim(q_emb, doc_emb)[0]
  top= torch.topk(scores, k=top_k)
  result = []
  for score,idx in zip(top.values,top.indices):
    result.append((float(score),docs[int(idx)]))
  return result






In [ ]:
query1 = "대화"
print(f'{query1}')
for s,text in search(query1):
  print(f'{s} | {text}')


대화
0.3656215965747833 | 나는 사과를 먹으면서 너와 즐겁게 대화를 했어
0.24063296616077423 | 나는 너 만나 사과를 먹으면서 너에게 잘못했던 문제에 대해서 사과를 했어
